In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3  # GoogLeNet alternative
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os
import zipfile
import shutil
from google.colab import files

# Upload and Extract Dataset ZIP
uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("dataset")

base_dataset_dir = "dataset"
extracted_folders = os.listdir(base_dataset_dir)

for folder in extracted_folders:
    folder_path = os.path.join(base_dataset_dir, folder)
    if os.path.isdir(folder_path):
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            target_path = os.path.join(base_dataset_dir, subfolder)
            if os.path.isdir(subfolder_path) and not os.path.exists(target_path):
                shutil.move(subfolder_path, base_dataset_dir)

# Remove empty directories
for folder in extracted_folders:
    folder_path = os.path.join(base_dataset_dir, folder)
    if os.path.isdir(folder_path) and not os.listdir(folder_path):
        os.rmdir(folder_path)

Saving GarbageClassification.zip to GarbageClassification.zip


In [8]:
# === Configuration ===
IMG_SIZE = (299, 299)
BATCH_SIZE = 32

# === Data Preparation ===
def load_dataset(directory, batch_size=BATCH_SIZE):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    # === Dataset Splitting ===
    train_generator = datagen.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

# === Load Dataset ===
dataset_dir = "dataset"
train_generator, val_generator = load_dataset(dataset_dir)

# === Summary Info ===
print("\n===== Dataset Summary =====")
print(f"Using {train_generator.samples} images for training")
print(f"Using {val_generator.samples} images for validation")
print("Class indices:", train_generator.class_indices)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.

===== Dataset Summary =====
Using 2024 images for training
Using 503 images for validation
Class indices: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
